In [4]:
import pandas as pd
import numpy as np
import requests
import xml.etree.ElementTree as ET


In [2]:

# Setup the URL and query parameters
url = "https://apis.data.go.kr/1741000/rnPpltnHhStus/selectRnPpltnHhStus"
params = {
    "serviceKey": "vV20MXMDsAE3CziS9d2N5S/80nDHRjStgsI9rzRFreL45qw0NIhKyiN/Mj2IBfm0KqGgSFaEuimoggy050xzAQ==",
    "roadNmCd": "111404103292",
    "srchFrYm": "202404",
    "srchToYm": "202404",
    "regSeCd": "1",
    "numOfRows": "100"
}

# Send the GET request
response = requests.get(url, params=params)

In [6]:
root = ET.fromstring(response.content)
data = []

# XML에서 필요한 정보 추출 (예: item 태그 내의 데이터)
for item in root.findall('.//item'):
    record = {}
    for child in item:
        record[child.tag] = child.text
    data.append(record)

# 데이터 프레임 생성
df = pd.DataFrame(data)

# 데이터 프레임 출력
df


,statsYm,roadNmCd,ctpvNm,sggNm,roadNm,totNmprCnt,hhCnt,hhNmpr,maleNmprCnt,femlNmprCnt,maleFemlRate
0,202404,111402005001,서울특별시,중구,세종대로,69,68,1.01,57,12,4.75
1,202404,111402100001,서울특별시,중구,삼일대로,2,2,1.0,1,1,1.0
2,202404,111403000008,서울특별시,중구,통일로,461,294,1.57,249,212,1.17
3,202404,111403005001,서울특별시,중구,동호로,483,336,1.44,208,275,0.76
4,202404,111403005002,서울특별시,중구,무교로,4,3,1.33,2,2,1.0
...,...,...,...,...,...,...,...,...,...,...,...
95,202404,111404103072,서울특별시,중구,다산로4길,112,78,1.44,52,60,0.87
96,202404,111404103073,서울특별시,중구,다산로5길,296,177,1.67,145,151,0.96
97,202404,111404103074,서울특별시,중구,다산로6길,460,240,1.92,209,251,0.83
98,202404,111404103075,서울특별시,중구,다산로8길,341,189,1.8,155,186,0.83


In [8]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd

def fetch_data(page_no):
    """ API에서 특정 페이지의 데이터를 가져오는 함수 """
    response = requests.get('https://apis.data.go.kr/1741000/rnPpltnHhStus/selectRnPpltnHhStus', params={
        'serviceKey': 'vV20MXMDsAE3CziS9d2N5S/80nDHRjStgsI9rzRFreL45qw0NIhKyiN/Mj2IBfm0KqGgSFaEuimoggy050xzAQ==',
        'roadNmCd': '111404103292',
        'srchFrYm': '202404',
        'srchToYm': '202404',
        'regSeCd': '1',
        'numOfRows': '100',
        'pageNo': page_no
    })
    return response.content

def parse_xml(xml_data):
    """ XML 데이터를 파싱하여 데이터 프레임에 추가하는 함수 """
    root = ET.fromstring(xml_data)
    data = []
    for item in root.findall('.//item'):
        record = {child.tag: child.text for child in item}
        data.append(record)
    return data

def main():
    # 첫 페이지 데이터를 불러와서 전체 항목 수와 페이지 수 계산
    first_page_xml = fetch_data(1)
    first_page_data = parse_xml(first_page_xml)
    total_count = int(ET.fromstring(first_page_xml).find('.//totalCount').text)
    rows_per_page = len(first_page_data)
    total_pages = (total_count + rows_per_page - 1) // rows_per_page

    # 모든 페이지 데이터 불러오기
    all_data = first_page_data
    for page_no in range(2, total_pages + 1):
        xml_data = fetch_data(page_no)
        all_data.extend(parse_xml(xml_data))

    # 데이터 프레임 생성 및 출력
    df = pd.DataFrame(all_data)
    return df
if __name__ == "__main__":
    df=main()


In [9]:
df

,statsYm,roadNmCd,ctpvNm,sggNm,roadNm,totNmprCnt,hhCnt,hhNmpr,maleNmprCnt,femlNmprCnt,maleFemlRate
0,202404,111402005001,서울특별시,중구,세종대로,69,68,1.01,57,12,4.75
1,202404,111402100001,서울특별시,중구,삼일대로,2,2,1.0,1,1,1.0
2,202404,111403000008,서울특별시,중구,통일로,461,294,1.57,249,212,1.17
3,202404,111403005001,서울특별시,중구,동호로,483,336,1.44,208,275,0.76
4,202404,111403005002,서울특별시,중구,무교로,4,3,1.33,2,2,1.0
...,...,...,...,...,...,...,...,...,...,...,...
375,202404,111404103401,서울특별시,중구,필동로8가길,76,35,2.17,44,32,1.38
376,202404,111404103402,서울특별시,중구,필동로8길,490,273,1.79,239,251,0.95
377,202404,111404103405,서울특별시,중구,후암로58길,19,19,1.0,18,1,18.0
378,202404,111404103406,서울특별시,중구,후암로60길,227,223,1.02,191,36,5.31
